# Tracer Advection Stencil

## Start a parallel cluster

In [1]:
from functions import show_clusters

import ipyparallel as ipp

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()


Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [2]:
%autopx

%autopx enabled


## Input parameters

In [421]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import numpy as np
import importlib

importlib.reload(func)

mpi_comm = MPI.COMM_WORLD

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 101, 101, 80, 3

dt_acoustic = 300.
advection_timestep = 3600.
nSteps_advection = 2000
print_advectionProgress = True
plot_output = True
figure_everyNsteps = 50

tracer_archive_frequency = figure_everyNsteps

grid_type = 0
hord = 6

pressure_base = 10
smoke_base = 1
smoke_center_tile = 0
density = 1

In [422]:
%%capture
dimensions, units, origins, rank = func.define_dimensionsUnitsOrigins(nx, ny, nz, nhalo, mpi_comm)
smoke_dict = {'center_tile': smoke_center_tile, 'rank': rank, 'smoke_base': smoke_base}
pressure_dict = {'pressure_base': pressure_base}
fvt_dict = {'grid_type': grid_type, 'hord': hord}

configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)
initialState = func.create_initialState_testCase1(configuration['grid_data'], dimensions, units, origins, backend, smoke_dict, pressure_dict)
tracAdv_data, tracAdv = func.prepare_everythingForAdvection(configuration, initialState['uC'], initialState['vC'], initialState['delp'], {'smoke': initialState['smoke']}, density, dimensions, units, origins, backend, fvt_dict, dt_acoustic=dt_acoustic)
tracAdv_dataInit = cp.deepcopy(tracAdv_data)

## Advection loop

In [423]:
tracer_archive = [cp.deepcopy(tracAdv_data['tracers']['smoke'])]
timestep_archive = [0]
for step in range(nSteps_advection):    
    if (rank == 0) and (step+1)%10 == 0 and print_advectionProgress:
        print('Step: %d' % (step+1))
    #tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep, mpi_rank=rank)
    tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep)

    if (step+1)%tracer_archive_frequency == 0:
        tracer_archive.append(cp.deepcopy(tracAdv_data['tracers']['smoke']))
        timestep_archive.append(step+1)

[stdout:0] Step: 10
Step: 20
Step: 30
Step: 40
Step: 50
Step: 60
Step: 70
Step: 80
Step: 90
Step: 100
Step: 110
Step: 120
Step: 130
Step: 140
Step: 150
Step: 160
Step: 170
Step: 180
Step: 190
Step: 200
Step: 210
Step: 220
Step: 230
Step: 240
Step: 250
Step: 260
Step: 270
Step: 280
Step: 290
Step: 300
Step: 310
Step: 320
Step: 330
Step: 340
Step: 350
Step: 360
Step: 370
Step: 380
Step: 390
Step: 400
Step: 410
Step: 420
Step: 430
Step: 440
Step: 450
Step: 460
Step: 470
Step: 480
Step: 490
Step: 500
Step: 510
Step: 520
Step: 530
Step: 540
Step: 550
Step: 560
Step: 570
Step: 580
Step: 590
Step: 600
Step: 610
Step: 620
Step: 630
Step: 640
Step: 650
Step: 660
Step: 670
Step: 680
Step: 690
Step: 700
Step: 710
Step: 720
Step: 730
Step: 740
Step: 750
Step: 760
Step: 770
Step: 780
Step: 790
Step: 800
Step: 810
Step: 820
Step: 830
Step: 840
Step: 850
Step: 860
Step: 870
Step: 880
Step: 890
Step: 900
Step: 910
Step: 920
Step: 930
Step: 940
Step: 950
Step: 960
Step: 970
Step: 980
Step: 990
Step: 10

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

Tracer advection -- updating fluxes?

In [424]:
%%capture
if plot_output:
    lon, lat = func.get_lonLat_agrid(configuration['grid_data'], dimensions, units, origins, backend)
    lon_global = configuration['communicator'].gather(lon)
    lat_global = configuration['communicator'].gather(lat)
    smoke_global = []
    for step in range(len(timestep_archive)):
        smoke_global.append(configuration['communicator'].gather(tracer_archive[step]))

    saveDir = '/home/ajdas/pace/advection_test/figs/'
    fOut = saveDir + 'advection_step_'

    if rank == 0:
        smoke_global = np.squeeze(np.stack(smoke_global))
        for plotStep in range(len(timestep_archive)):
            print(plotStep, timestep_archive[plotStep])
            func.plot_projection_field(lon_global.data, lat_global.data, smoke_global[plotStep], 
                                       cmap='viridis', vmin=0, vmax=1, units='kg', 
                                       title='Smoke after %s advection steps' % timestep_archive[plotStep], 
                                       fSave=fOut+'{:03d}'.format(timestep_archive[plotStep]) + '.png')

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [ ]:
%autopx

In [ ]:
cluster.shutdown()